In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [94]:
data = pd.read_csv('../data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [95]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [119]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [120]:
data_train.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [121]:
def weighted_random_recommendation(items_weights, n):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
        n - количество элементов в списке рекомендаций
    """
    items = np.array(items_weights['item_id'])
    it_weights = np.array(items_weights['weight'])
    recs = random.choices(items, weights = it_weights, k = n)

    
    return recs

In [122]:
%%time

items_weights = data_train.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value']/items_weights['sales_value'].sum()
items_weights.drop(['sales_value'], axis = 1, inplace = True)
items_weights.head(3)

CPU times: user 107 ms, sys: 28.1 ms, total: 135 ms
Wall time: 138 ms


,item_id,weight
0,25671,2.969296e-06
1,26081,1.403822e-07
2,26093,2.254623e-07


In [123]:
%time
result['weight_rand_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.01 µs


,user_id,actual,weight_rand_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9419444, 989101, 821976, 9526886, 930917]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9526572, 1029743, 958697, 1126899, 920191]"


In [124]:
weighted_random_recommendation(items_weights, n=7)

[933014, 965267, 1005186, 846282, 10456457, 1103618, 13002975]

Пробовала вычислять веса по разному и по разным столбцам
Например, есть вычислять так #items_weights['weight'] = np.log(items_weights['sales_value']) 
то, у весов получается слишком большой разброс и в выборку попадает практически одно значение
Можно попробовать убирать из списка товаров те, которые уже попали в выборку

Поэтому остановилась на варианте рассчета весов как долю от общих продаж

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [125]:
result = pd.read_csv('../predictions/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[1011736, 1016274, 51226, 9219941, 932166]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1622526, 1846997, 928009, 970747, 15717358]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [126]:
result['weight_rand_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

In [127]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weight_rand_rec
0,1,[ 821867 834484 856942 865456 889248 ...,"[1011736, 1016274, 51226, 9219941, 932166]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[6534178, 926449, 947798, 1084177, 12262778]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1622526, 1846997, 928009, 970747, 15717358]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[9677543, 8290421, 1093413, 977374, 1051258]"


В связи с тем, что после сохранения и открытия файла csv изменились типы данных, выполним дополнительные преобразования

In [128]:
result['random_recommendation'] = result['random_recommendation'].map(lambda x: x[1:-1].split(', '))
result['popular_recommendation'] = result['popular_recommendation'].map(lambda x: x[1:-1].split(', '))
result['itemitem'] = result['itemitem'].map(lambda x: x[1:-1].split(', '))
result['cosine'] = result['cosine'].map(lambda x: x[1:-1].split(', '))
result['tfidf'] = result['tfidf'].map(lambda x: x[1:-1].split(', '))
result['own_purchases'] = result['own_purchases'].map(lambda x: x[1:-1].split(', '))

In [129]:
data_test = data_test.astype({'item_id': str})

In [130]:
result2 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result2.columns=['user_id', 'actual']
result['actual'] = result2['actual']

In [131]:
type(result['actual'].iloc[0][0])

str

In [132]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [133]:
def precision_at_k_column(recommended_list, bought_list, k=5):
    a = []
    for i in range(recommended_list.shape[0]):
        a.append(precision_at_k(recommended_list.iloc[i], bought_list[i], k=5))
    return np.mean(a)

In [134]:
a = precision_at_k_column(result['random_recommendation'], result['actual'], k=5)
print('средний precision_at_k для random_recommendation {}'.format(a))
b = precision_at_k_column(result['popular_recommendation'], result['actual'], k=5)
print('средний precision_at_k для popular_recommendation {}'.format(b))
d = precision_at_k_column(result['itemitem'], result['actual'], k=5)
print('средний precision_at_k для itemitem {}'.format(d))
e = precision_at_k_column(result['cosine'], result['actual'], k=5)
print('средний precision_at_k для cosine {}'.format(e))
f = precision_at_k_column(result['tfidf'], result['actual'], k=5)
print('средний precision_at_k для tfidf {}'.format(f))
g = precision_at_k_column(result['own_purchases'], result['actual'], k=5)
print('средний precision_at_k для own_purchases {}'.format(g))
h = precision_at_k_column(result['weight_rand_rec'], result['actual'], k=5)
print('средний precision_at_k для weight_rand_rec {}'.format(g))

средний precision_at_k для random_recommendation 0.000881488736532811
средний precision_at_k для popular_recommendation 0.15523996082272282
средний precision_at_k для itemitem 0.13692458374143
средний precision_at_k для cosine 0.1329089128305583
средний precision_at_k для tfidf 0.13898139079333988
средний precision_at_k для own_purchases 0.17969311132876264
средний precision_at_k для weight_rand_rec 0.17969311132876264


Лучший результат получился для own_purchases и weight_rand_rec

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [135]:
data = pd.read_csv('../data/retail_train.csv')

In [136]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()

In [137]:
np.sort(popularity.sales_value)[::-1]

array([467993.62000001,  42645.75      ,  37981.91      , ...,
            0.        ,      0.        ,      0.        ])

In [138]:
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [139]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [141]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [142]:
popularity = popularity.reset_index().sort_values(by=['sales_value', 'index'], ascending=False).set_index('index')
popularity = popularity[:5000]

In [143]:
popularity.shape

(5000, 2)

In [144]:
%%time

items = popularity.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 223 ms, sys: 4.69 ms, total: 228 ms
Wall time: 250 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[912112, 7155012, 985605, 1069047, 892862]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[911411, 982393, 1010440, 1015386, 949142]"


In [145]:
%%time
popular_recs = popularity.item_id.unique()[:5]
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 2.84 ms, sys: 687 µs, total: 3.52 ms
Wall time: 3.02 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[912112, 7155012, 985605, 1069047, 892862]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[911411, 982393, 1010440, 1015386, 949142]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [146]:
items_weights = popularity.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['weight'] = items_weights['sales_value']/items_weights['sales_value'].sum()
items_weights.drop(['sales_value'], axis = 1, inplace = True)

In [147]:
result['weight_rand_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weight_rand_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[912112, 7155012, 985605, 1069047, 892862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1029743, 1044078, 1014732, 848029, 1075470]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[911411, 982393, 1010440, 1015386, 949142]","[6534178, 6533889, 1029743, 6534166, 1082185]","[862860, 1081177, 9296837, 837107, 910109]"


In [148]:
type(result['random_recommendation'][7][0])

int

In [149]:
a = precision_at_k_column(result['random_recommendation'], result['actual'], k=5)
print('средний precision_at_k для random_recommendation {}'.format(a))
b = precision_at_k_column(result['popular_recommendation'], result['actual'], k=5)
print('средний precision_at_k для popular_recommendation {}'.format(b))
c = precision_at_k_column(result['weight_rand_rec'], result['actual'], k=5)
print('средний precision_at_k для weight_rand_rec {}'.format(c))

средний precision_at_k для random_recommendation 0.006562193927522038
средний precision_at_k для popular_recommendation 0.15523996082272282
средний precision_at_k для weight_rand_rec 0.034280117531831536


Результаты на всех товарах средний precision_at_k: random_recommendation 0.00088
                                                   popular_recommendation 0.15523 
                                                   weight_rand_rec 0.179693
Вывод - 
для random_recommendation результат улучшился
для popular_recommendation остался таким же, так как выборка одна и та же
для  weight_rand_rec результат ухудшился, так как в выборку стало попадать меньше товаров и покупают не всегда самые популярные товары

**Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾**

In [150]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [151]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [152]:
# Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [154]:
np.sort(data.item_id.unique())

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556])

In [155]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [156]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.48 s, sys: 67.1 ms, total: 1.54 s
Wall time: 1.76 s


In [157]:
recs

[(2381, 78679.0),
 (3408, 72173.0),
 (2148, 57995.0),
 (3947, 17272.0),
 (3587, 14417.0)]

In [158]:
[id_to_itemid[rec[0]] for rec in recs]

[999999, 1082185, 981760, 1127831, 1098066]

In [159]:
%%time

result['itemitem_k5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 66.6 ms, sys: 10.5 ms, total: 77.2 ms
Wall time: 79 ms


изменим количество соседей на 10

In [160]:
%%time

model2 = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей
model2.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs2 = model2.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 1.52 s, sys: 29.4 ms, total: 1.55 s
Wall time: 1.56 s


In [161]:
%%time

result['itemitem_k10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model2.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 80.2 ms, sys: 3.53 ms, total: 83.7 ms
Wall time: 84.4 ms


изменим количество соседей на 15

In [163]:
%%time
model3 = ItemItemRecommender(K=15, num_threads=4) # K - кол-во билжайших соседей
model3.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs3 = model3.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

result['itemitem_k15'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model3.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])




CPU times: user 1.59 s, sys: 21.8 ms, total: 1.62 s
Wall time: 1.63 s


In [164]:
result.head(10)

,user_id,actual,random_recommendation,popular_recommendation,weight_rand_rec,itemitem_k5,itemitem_k10,itemitem_k15
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[912112, 7155012, 985605, 1069047, 892862]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1029743, 1044078, 1014732, 848029, 1075470]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[911411, 982393, 1010440, 1015386, 949142]","[6534178, 6533889, 1029743, 6534166, 1082185]","[862860, 1081177, 9296837, 837107, 910109]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1108940, 1101589, 937413, 1037451, 6979526]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 968072, 819308, 1070845, 1075007]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 840361, 995242]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[834932, 970160, 930918, 1124971, 843744]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 863030, 941295, 866211, 6534178]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 1098066]","[999999, 1082185, 981760, 995242, 840361]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6979128, 9829205, 8090509, 1010259, 857348]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1106523, 942525, 831712, 7025362, 1139853]","[999999, 1082185, 981760, 1127831, 1098066]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 995242, 840361]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[12484491, 1120362, 5567601, 1018740, 1062002]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12262778, 1092835, 1103691, 868764, 906838]","[999999, 1082185, 981760, 995242, 1098066]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 995242]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[961791, 877358, 1132178, 6391557, 927291]","[6534178, 6533889, 1029743, 6534166, 1082185]","[910473, 9677236, 1029743, 827683, 921243]","[999999, 1082185, 981760, 1098066, 1127831]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 995242, 840361]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[10149640, 1117899, 864900, 1108296, 822785]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1092120, 9856740, 1065538, 835028, 1003487]","[999999, 1082185, 981760, 1098066, 995242]","[999999, 1082185, 981760, 1098066, 826249]","[999999, 1082185, 981760, 1098066, 995242]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[876984, 1013407, 1056509, 9396705, 884945]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1010573, 1119761, 6534178, 939636, 6534178]","[999999, 1082185, 981760, 1127831, 995242]","[999999, 1082185, 981760, 995242, 840361]","[999999, 1082185, 981760, 995242, 840361]"
9,16,"[1062973, 1082185, 13007710]","[1089392, 1092363, 954495, 1034868, 12487816]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6534178, 1096099, 1043590, 1081177, 930118]","[999999, 1082185, 981760, 995242, 1029743]","[999999, 1082185, 981760, 840361, 1098066]","[999999, 1082185, 981760, 995242, 840361]"


In [165]:
e = precision_at_k_column(result['itemitem_k5'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k5 {}'.format(e))
f = precision_at_k_column(result['itemitem_k10'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k10 {}'.format(f))
g = precision_at_k_column(result['itemitem_k15'], result['actual'], k=5)
print('средний precision_at_k для itemitem_k15 {}'.format(g))

средний precision_at_k для itemitem_k5 0.13692458374143
средний precision_at_k для itemitem_k10 0.1509304603330069
средний precision_at_k для itemitem_k15 0.15318315377081293


Вывод - качество улучшилось при увеличении соседей